# Startup 
Code to be include at the beginning of each Julia script.
IVER is the version defined in the batch file.

In [1]:
if !isdefined(Main,:IVER) 
    if "IVER" in keys(ENV)
        IVER = parse(Int,ENV["IVER"])
        println("-----***** Version $IVER *****-----")
    else
        IVER = 3
        println("-----***** IVER not specified => Default version $IVER *****-----")
    end
end

-----***** IVER not specified => Default version 3 *****-----


Define directories and call packages.

In [2]:
dir_module = joinpath(pwd(),"src")
dir_mom = joinpath(pwd(),"../../data/moments")
dir_calib = joinpath(pwd(),"calibrated")

########### main module
using NBInclude
@nbinclude(joinpath(dir_module,"Opmismatch.ipynb"))
using Main.Opmismatch

########### other packages
using DataFrames:DataFrame
using Distributions:mean, std
using LinearAlgebra:diag, Diagonal
using Roots:fzero
using CSV

## Empirical moments

In [3]:
N = Hyper(22,22)
emp, jdr = let
    
    df_stats = DataFrame(CSV.read(joinpath(dir_mom,"stats.csv")))
    df_rates = DataFrame(CSV.read(joinpath(dir_mom,"rates.csv")))
    (shUU,shUE,shEU,shEEnew,shEEsame) = share_transit(df_rates,N)

    mom1, mom4 = def_mom14(df_stats,N)
    (mom2,mom3,jdr) = def_mom23(shUE, shEEnew+shEEsame, N)
    (Moments(mom1,mom2,mom3,mom4), jdr )
end
;

## Initial values

In [4]:
eta_vec = [0.25,0.25,0.25,0.5,0.5,0.5,0.75,0.75,0.75]
beta_vec = [0.25,0.5,0.75,0.25,0.5,0.75,0.25,0.5,0.75]

M_init = let
    nu = fill(NaN,N.I) #emp.mom4 ./ sum(emp.mom4)
    
    h = 0.4 * weightedmean(emp.mom1,emp.mom4)
    y = zeros(N.I,N.J)  
    #fill(1.1 * h,N.I,N.J) 
    #for j0 in 1:N.J
    #    y[j0,j0] = emp.mom1[j0]*3
    #end
    k = fill(NaN,N.J) # fill(5.,N.J)
    r = fzero(x->exp.(-12*x)-1/(1+0.05), 0, 1) # 5% annual interest rate
    delta = jdr 
    eta = eta_vec[IVER]
    beta = beta_vec[IVER]
    es = NaN  # 0.5
    xi = 0. # 1.
    Para(nu=nu,r=r,delta=delta,eta=eta,beta=beta,y=y,h=h,k=k,es=es,xi=xi)
end


println("-----*****  eta=$(M_init.eta) and beta=$(M_init.beta) *****-----")

-----*****  eta=0.25 and beta=0.75 *****-----


The current tax system is modeled with a fix wage tax of 30\%, wich means $\tau^w=3/7$.

In [5]:
## define taxes at 0
Notax = Taxes(0.,0.,0.,zeros(N.J),zeros(N.J))

RTax = let
    T0 = Taxes(3/7 ,0.,0., zeros(N.J), zeros(N.J)) ## to be compensated
    comp_Taxes(T0,M_init)
end

Taxes{Float64}(0.42857142857142855, 0.33214285714285713, -0.10714285714285715, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

We define the bounds for the calibration.

In [6]:
## parameters for the calibration
PAR = let
    wgt = build_weights(emp) 
    lbd = Bounds(nuc=0., y=M_init.h, alpha=0., theta=1e-25, es=0.01, xi=0.01 )
    ubd = Bounds(nuc=1000., y=100., alpha=1-1e-10, theta=1e10, es=0.99, xi=10.)
    ParCalib(emp,wgt,lbd,ubd)
end
;